# 東京家賃予測 - Deep Learning モデル可視化

このノートブックでは、訓練されたディープラーニングモデルの詳細な可視化と分析を行います。

## 可視化の内容
1. ネットワークアーキテクチャ
2. 区の埋め込み（インタラクティブ3D）
3. 区の類似度分析
4. Attention機構の可視化
5. 訓練プロセスの分析
6. 予測誤差の分析

## 1. ライブラリのインポート

In [ ]:
# 日本語フォント設定
from rent_utils import setup_japanese_font
setup_japanese_font()

import matplotlib.pyplot as plt
import matplotlib.font_manager as fmt


print("✅ 日本語フォント設定完了")

import matplotlib.pyplot as plt
import matplotlib.font_manager as fmt
import pandas as pd
import numpy as np
import torch
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from rent_dl_visualize import NeuralNetworkVisualizer

# プロット設定
plt.rcParams['figure.dpi'] = 100
sns.set_style('whitegrid')

print("ライブラリのインポート完了")

## 2. ビジュアライザーの初期化

In [ ]:
# Attentionモデル用のビジュアライザー
visualizer = NeuralNetworkVisualizer(
    model_type='attention',
    model_path='rent_prediction_model_attention.pth',
    data_path='tokyo_rent_data_v2.csv'
)

print("ビジュアライザー準備完了!")

## 3. ネットワークアーキテクチャの可視化

In [ ]:
# ネットワーク構造、パラメータ分布、埋め込み、モデル情報を可視化
visualizer.visualize_architecture()

## 4. 区の埋め込み - インタラクティブ3D可視化

In [ ]:
# 3D PCA投影のインタラクティブプロット
visualizer.visualize_embeddings_interactive()

## 5. 区の類似度分析

In [ ]:
# 埋め込みベクトルを取得
embeddings = visualizer.model.ward_embedding.weight.detach().cpu().numpy()
ward_names = visualizer.preprocessor.label_encoders['区'].classes_

# 類似度マトリクスを可視化
visualizer.plot_similarity_matrix(embeddings, ward_names)

In [ ]:
# コサイン類似度を計算
similarity_matrix = cosine_similarity(embeddings)

# 最も類似している区のペアを探す
similar_pairs = []
for i in range(len(ward_names)):
    for j in range(i+1, len(ward_names)):
        similar_pairs.append((
            ward_names[i],
            ward_names[j],
            similarity_matrix[i, j]
        ))

similar_pairs.sort(key=lambda x: x[2], reverse=True)

print("\n最も類似している区のペア (Top 10):")
print("=" * 60)
for i, (ward1, ward2, sim) in enumerate(similar_pairs[:10], 1):
    print(f"{i:2d}. {ward1:8s} ↔ {ward2:8s}  類似度: {sim:.4f}")

## 6. Attention機構の可視化

In [ ]:
# Attentionウェイトのヒートマップ
visualizer.visualize_attention_weights()

In [ ]:
# 区別のAttention重要度分析
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ward_avg_prices = visualizer.df.groupby('区')['家賃_円'].mean()

attention_scores = []
for i, ward in enumerate(ward_names):
    ward_idx = torch.LongTensor([i]).to(device)
    with torch.no_grad():
        ward_emb = visualizer.model.ward_embedding(ward_idx)
        attention = visualizer.model.attention(ward_emb)
        score = attention.sum().item()
    
    attention_scores.append({
        '区': ward,
        'Attention Score': score,
        '平均家賃': ward_avg_prices[ward]
    })

attention_df = pd.DataFrame(attention_scores)
attention_df = attention_df.sort_values('Attention Score', ascending=False)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Attentionスコアランキング
axes[0].barh(attention_df['区'][:15], attention_df['Attention Score'][:15], color='steelblue')
axes[0].set_xlabel('Attention Score')
axes[0].set_title('Attention Score by Ward (Top 15)', fontweight='bold')
axes[0].invert_yaxis()

# Attentionスコアと平均家賃の関係
axes[1].scatter(attention_df['平均家賃'], attention_df['Attention Score'], 
                s=100, alpha=0.6, c='coral')
for i, row in attention_df.iterrows():
    if row['区'] in ['港区', '千代田区', '足立区', '葛飾区']:
        axes[1].annotate(row['区'], (row['平均家賃'], row['Attention Score']),
                         xytext=(5, 5), textcoords='offset points', fontsize=9)
axes[1].set_xlabel('Average Rent (¥)')
axes[1].set_ylabel('Attention Score')
axes[1].set_title('Attention Score vs Average Rent', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 相関係数
correlation = attention_df['平均家賃'].corr(attention_df['Attention Score'])
axes[1].text(0.05, 0.95, f'Correlation: {correlation:.3f}',
             transform=axes[1].transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('attention_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("可視化保存: attention_analysis.png")

## 7. t-SNE による埋め込み可視化

In [ ]:
# t-SNE 2D投影
tsne = TSNE(n_components=2, random_state=42, perplexity=8)
embeddings_tsne = tsne.fit_transform(embeddings)

colors = [ward_avg_prices[ward] for ward in ward_names]

plt.figure(figsize=(14, 10))
scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1],
                      c=colors, cmap='RdYlBu_r', s=300, alpha=0.7, edgecolors='black')

# すべての区名を表示
for i, ward in enumerate(ward_names):
    plt.annotate(ward, (embeddings_tsne[i, 0], embeddings_tsne[i, 1]),
                 fontsize=9, ha='center', va='center', fontweight='bold')

plt.xlabel('t-SNE Dimension 1', fontsize=12)
plt.ylabel('t-SNE Dimension 2', fontsize=12)
plt.title('Ward Embeddings (t-SNE 2D Projection)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Average Rent (¥)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('ward_embeddings_tsne.png', dpi=300, bbox_inches='tight')
plt.show()

print("可視化保存: ward_embeddings_tsne.png")

## 8. 訓練プロセスの分析（データがある場合）

In [ ]:
# 訓練履歴をロード（訓練ノートブックから保存されている場合）
try:
    # 訓練履歴がある場合
    train_history = torch.load('training_history.pth', weights_only=False)
    train_losses = train_history['train_losses']
    val_losses = train_history['val_losses']
    
    visualizer.visualize_training_analysis(train_losses, val_losses)
except FileNotFoundError:
    print("訓練履歴ファイルが見つかりません。")
    print("rent_dl_train.ipynbで訓練を実行してください。")

## 9. 予測誤差の分析

In [ ]:
# テストデータで予測を実行
from torch.utils.data import DataLoader
from rent_data import TokyoRentDataset
from sklearn.model_selection import train_test_split

# データ準備（訓練ノートブックと同じ処理）
df = visualizer.df
feature_cols = ['区_encoded', '建物構造_encoded', '建物タイプ_encoded',
                '部屋サイズ_m2', '駅距離_分', '築年数_年', '区_avg_price']

from rent_data import RentDataPreprocessor
preprocessor = RentDataPreprocessor()
df_processed = preprocessor.fit_transform(df)

X = df_processed[feature_cols].values
y = df['家賃_円'].values / 10000

# データ分割
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# テストデータの区情報を保存
test_indices = X_test[:, 0].astype(int)
test_wards = [ward_names[idx] for idx in test_indices]

test_dataset = TokyoRentDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 予測実行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
visualizer.model.eval()

predictions = []
actuals = []

with torch.no_grad():
    for batch_features, batch_targets in test_loader:
        batch_features = batch_features.to(device)
        
        ward_idx = batch_features[:, 0].long()
        structure_idx = batch_features[:, 1].long()
        type_idx = batch_features[:, 2].long()
        numeric_features = batch_features[:, 3:6]
        ward_avg_price = batch_features[:, 6]
        
        outputs = visualizer.model(ward_idx, structure_idx, type_idx, 
                                    numeric_features, ward_avg_price)
        if isinstance(outputs, tuple):
            outputs = outputs[0]
        
        predictions.extend(outputs.cpu().numpy().flatten() * 10000)
        actuals.extend(batch_targets.numpy() * 10000)

predictions = np.array(predictions)
actuals = np.array(actuals)
errors = actuals - predictions
percent_errors = (errors / actuals) * 100

print("予測完了!")

In [ ]:
# 誤差分析の可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 誤差の分布
axes[0, 0].hist(errors, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Error (¥)', fontsize=11)
axes[0, 0].set_ylabel('Frequency', fontsize=11)
axes[0, 0].set_title('Distribution of Prediction Errors', fontweight='bold')
axes[0, 0].text(0.05, 0.95, f'Mean: ¥{errors.mean():,.0f}\nStd: ¥{errors.std():,.0f}',
                transform=axes[0, 0].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 2. パーセント誤差の分布
axes[0, 1].hist(percent_errors, bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Percent Error (%)', fontsize=11)
axes[0, 1].set_ylabel('Frequency', fontsize=11)
axes[0, 1].set_title('Distribution of Percent Errors', fontweight='bold')
axes[0, 1].text(0.05, 0.95, f'Mean: {percent_errors.mean():.1f}%\nStd: {percent_errors.std():.1f}%',
                transform=axes[0, 1].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 3. 価格帯別の誤差
price_bins = [0, 80000, 100000, 120000, 150000, 200000, np.inf]
price_labels = ['<80k', '80-100k', '100-120k', '120-150k', '150-200k', '>200k']
price_categories = pd.cut(actuals, bins=price_bins, labels=price_labels)

error_by_price = pd.DataFrame({
    'Price Category': price_categories,
    'Absolute Error': np.abs(errors)
})

error_by_price.boxplot(column='Absolute Error', by='Price Category', ax=axes[1, 0])
axes[1, 0].set_xlabel('Price Range', fontsize=11)
axes[1, 0].set_ylabel('Absolute Error (¥)', fontsize=11)
axes[1, 0].set_title('Error by Price Range', fontweight='bold')
plt.sca(axes[1, 0])
plt.xticks(rotation=45)

# 4. 区別の平均誤差
error_by_ward = pd.DataFrame({
    'Ward': test_wards,
    'Absolute Error': np.abs(errors)
})

ward_error_avg = error_by_ward.groupby('Ward')['Absolute Error'].mean().sort_values(ascending=False)

axes[1, 1].barh(ward_error_avg.index[:15], ward_error_avg.values[:15], color='lightgreen')
axes[1, 1].set_xlabel('Average Absolute Error (¥)', fontsize=11)
axes[1, 1].set_title('Average Error by Ward (Top 15)', fontweight='bold')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.savefig('error_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("可視化保存: error_analysis.png")

In [ ]:
# 誤差統計サマリー
print("\n" + "=" * 60)
print("誤差分析サマリー")
print("=" * 60)
print(f"平均絶対誤差 (MAE):     ¥{np.abs(errors).mean():,.0f}")
print(f"二乗平均平方根誤差 (RMSE): ¥{np.sqrt((errors**2).mean()):,.0f}")
print(f"平均誤差:              ¥{errors.mean():,.0f}")
print(f"誤差の標準偏差:         ¥{errors.std():,.0f}")
print(f"\n平均パーセント誤差:     {percent_errors.mean():.2f}%")
print(f"パーセント誤差の標準偏差: {percent_errors.std():.2f}%")
print(f"\n最大過大評価:          ¥{errors.min():,.0f}")
print(f"最大過小評価:          ¥{errors.max():,.0f}")

## 完了!

すべての可視化が完了しました。生成されたファイル:
- `network_architecture_viz.png` - ネットワーク構造
- `ward_embeddings_3d.html` - 3D埋め込み（インタラクティブ）
- `ward_similarity_matrix.html` - 類似度マトリクス
- `attention_weights_viz.html` - Attentionウェイト
- `attention_analysis.png` - Attention分析
- `ward_embeddings_tsne.png` - t-SNE可視化
- `error_analysis.png` - 誤差分析